In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

np.random.seed(0)

In [ ]:
def get_pos_vel(itime=0):
    """Return True Position and Measured Velocity.
        - State space model:
            * x_{k+1} = A x_{k} + w_{k}
            * z_{k} = H x_{k} + v_{k}
        - System Model:
            (1) Initial condition:
                y_pos_true_{0} = 0 [m]
                y_vel_true_{0} = 80 [m/s]
                w_{k} = N(0, 10)
                v_{k} = N(0, 10)
            (2) System condition:
                A (state transition model)              = [[1, dt]
                                                           [0, 1]]
                H (observation model)                   = [[1, 0]]
                Q (covariance of the process noise)     = [[1, 0],
                                                           [0, 3]]
                R (covariance of the observation noise) = [[10]]
            (3) State space model:
                y_pos_true_{k+1} = y_pos_true_{k} + y_vel_true_{k} * dt
                y_vel_true_{k+1} = y_vel_true_{k}
                z_vel_{k} = y_vel_true_{k} + v_{k}
    """
    v_mean = 0
    v_std = np.sqrt(10)
    v = np.random.normal(v_mean, v_std)  # v: measurement noise.

    y_pos_true_0 = 0                                                # y_pos_true_0: initial position [m] 
    y_vel_true_0 = 80                                               # y_vel_true_0: initial velocity [m/s]
    y_pos_true = y_pos_true_0 + y_vel_true_0 * (itime * time_step)  # y_pos_true: true position. 
    y_vel_true = y_vel_true_0                                       # y_vel_true: true velocity.
    z_vel_meas = y_vel_true + v                                     # z_pos_meas: measured position (observable) 

    return y_pos_true, z_vel_meas

In [ ]:
def kalman_filter(z_meas, x_esti, P):
    """Estimate position using a kalman filter."""
    # (1) Prediction.
    x_pred = A @ x_esti
    P_pred = A @ P @ A.T + Q

    # (2) Kalman Gain.
    K = P_pred @ H.T @ inv(H @ P_pred @ H.T + R)

    # (3) Eastimation.
    x_esti = x_pred + K @ (z_meas - H @ x_pred)

    # (4) Error Covariance.
    P = P_pred - K @ H @ P_pred

    return x_esti, P

In [ ]:
# Input parameters.
time_start = 0
time_end = 4
time_step = 0.1

In [ ]:
# Initialization for system model.
# Define matrix with rank 2 for matrix operation.
dt = time_step
A = np.array([[1, dt],
              [0, 1]])
H = np.array([[0, 1]])
Q = np.array([[1, 0],
              [0, 3]])
R = np.array([[10]])

# Initialization for estimation.
x_0 = np.array([0, 20])  # position and velocity
P_0 = 5 * np.eye(2)

In [ ]:
time = np.arange(time_start, time_end, time_step)
n_samples = len(time)

In [ ]:
z_vel_meas_save = np.zeros(n_samples)
y_pos_true_save = np.zeros(n_samples)
x_pos_esti_save = np.zeros(n_samples)
x_vel_esti_save = np.zeros(n_samples)

In [ ]:
x_esti, P = None, None
for i in range(n_samples):
    y_pos_true, z_vel_meas = get_pos_vel(i)
    if i == 0:
        x_esti, P = x_0, P_0
    else:
        x_esti, P = kalman_filter(z_vel_meas, x_esti, P)

    z_vel_meas_save[i] = z_vel_meas
    y_pos_true_save[i] = y_pos_true
    x_pos_esti_save[i] = x_esti[0]
    x_vel_esti_save[i] = x_esti[1]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

plt.subplot(1, 2, 1)
plt.plot(time, x_vel_esti_save, 'bo-', label='Estimation (KF)')
plt.plot(time, z_vel_meas_save, 'r*--', label='Measurements', markersize=10)
plt.legend(loc='lower right')
plt.title('Velocity: Meas. v.s. Esti. (KF)')
plt.xlabel('Time [sec]')
plt.ylabel('Velocity [m/s]')

plt.subplot(1, 2, 2)
plt.plot(time, x_pos_esti_save, 'bo-', label='Estimation (KF)')
plt.plot(time, y_pos_true_save, 'g*--', label='True', markersize=10)
plt.legend(loc='upper left')
plt.title('Position: True v.s. Esti. (KF)')
plt.xlabel('Time [sec]')
plt.ylabel('Position [m]')
plt.savefig('png/vel2pos_kf.png')